In [5]:
import os
import numpy as np
import pandas as pd
from cloudvolume import CloudVolume
import json
from annotationframeworkclient import FrameworkClient
import nglui
from concurrent import futures
from pathlib import Path

## Uploading data to the annotation engine:
#### 1. Get a JSON state
#### 2. Upload neurons or synapses
---
##### Token for V1 chunkedgraph on dev: https://dev.zetta.ai/wclee/auth/api/v1/refresh_token
##### Token for annotation framework from here: https://api.zetta.ai/wclee/auth/api/v1/refresh_token
- These tokens are currently stored in ~/cloudvolume/secrets/authtokens.json, you should put them here too since this folder will be used by other things. 

#### Generate client object

In [56]:
with open(Path.home() / 'cloudvolume' / 'secrets'/'chunkedgraph-secret.json') as f:
        tokens = json.load(f)

# Tokens from above links passed as strings. 
dev_token = tokens['dev']
auth_token = tokens['api']

datastack_name = 'vnc_v0' # from https://api.zetta.ai/wclee/info/

client = FrameworkClient(
    datastack_name,
    server_address = "https://api.zetta.ai/wclee",
    auth_token = auth_token
)

#### Get the JSON state from a NG instance containing an annotation layer.
- JSON states are the primary means of managing things.
- When working in neuroglancer, to save the state of your workspace, press ctrl-shift-j. This will change the url with the /xxxxx being the JSON state ID. These states are stored in the JSON state service of the annotation framework. They are NOT EASY to look up, so here is a method for managing them.
- Define a state manager to keep track of our states. Save the state dataframe to the ~/.cloudvolume. We will put this in a module later. 

In [23]:
class StateManager: 
    
    ''' Class for keeping track of JSON states.'''
    def __init__(self,
                 filename=None,
                 token=None):
        
        self.directory = Path.home() / 'cloudvolume'
        if filename is None:
            self.filename = self.directory / 'json_states.csv'
        
        self.__initialize()
        
        
    def __initialize(self):
        # Check if the database exists, if not create a new one.
        fileEmpty =  os.path.exists(self.filename)
        if not fileEmpty:
            df = pd.DataFrame(columns=['state_id','description'])
            df.to_csv(self.filename,index=False)
        self.get_database()
        print(self.df) 
    
    def get_database(self):
        # Read database. 
        self.df  = pd.read_csv(self.filename)
        
 
    def add_state(self, state_id, description=None):
        
        filename = self.filename
        df = pd.DataFrame([{'state_id':state_id,'description':description}])
        df.to_csv(filename, mode='a', header=False,index=False, encoding = 'utf-8')
        self.get_database()
        return('state added')
 
    def get_state(self,state_id):
        return(client.state.get_state_json(state_id))
    
    def get_url(self,state):
        return('https://neuromancer-seung-import.appspot.com/?json_url=https://api.zetta.ai/json/' + str(state))
        

#### Look up the state ID you want, or add a state with `sm.add_state()`
#### Next, get the JSON state with `sm.get_state`

In [26]:
sm = StateManager()

             state_id     description
0  261002187096550146  MNs,10Bs,Claws


In [30]:
state = sm.get_state(sm.df.state_id[0])

#### Look at available layers
- If you had annotation layers in your state, marking either synapses or cells, they will be shown here. 

In [33]:
nglui.parser.layer_names(state)

['realigned_v1', 'vnc1_full_v3align_2', 'MNs', '10B', 'claws']

#### Define some methods for formatting schemas. 
##### We are going to use two schemas for now. 
1. The synapse schema for synapses.
    - To generate synapse entries, use an annotation layer that used line annotations with pt1 being the presynapse and pt2 being the postsynspse. 
2. The bound_tag schema for marking cells. 
    - To generate cell entreis, use an annotation layer that used point annotations, preferably on the soma (but not in the nucleus unless you merged the nucleus).

In [34]:
def soma_table_entries(state,layer_name = 'cells'):
    ''' Generate entries for a soma table using the bound_tag schema 
    Args:
    state: json, json state from get_json_state
    layer_name: name of layer containing soma coords
    ### Change this to points later. This is just for database copying because I used a stupid annotation
    
    Returns:
    entries: list, list of dict entries for the bound_tag schema'''
    
    cell_layer = nglui.parser.get_layer(state,layer_name)
    entries = []
    for i in cell_layer['annotations']:
        entry = {'tag': i['description'],
                 'pt': {'position': i['pointA']}}
        entries.append(entry)
    return(entries)
        

def upload_cells(cell_entries,table_name,description=None):
    ''' Upload cell entries to a soma dable using the bound_tag schema'
    Args:
    cell_entries: list, list of dicts from soma_table_entries
    table_name: str, table name to create. If it exists, it will populate the existing one. 
    description: str, description to supply if creating a new table. It will fail if you don't provide one. 
    
    ##TODO: Some sort of check for redundant synapses is necessary. We could make each upload need to go to a new table, but that seems absurd. 
'''
    
    try:
        client.annotation.create_table(table_name=table_name,
                                   schema_name='bound_tag',
                                   description=description)
    except:
        print('table exists')
        
    for i in cell_entries:
        try:
            client.annotation.post_annotation(table_name=table_name, data=[i])
        except:
            print('Fail',entry)



def get_synapses(state,synapse_layer='synapses'):
    ''' Get the synapse coordinates from a json state and return pre,post,center.
    Args:
        state: json,  json state from get_state_json 
        synapse_layer: str, name of layer containing synapses in the json state. Default is 'synapses'
    Returns:
        pre_pt,post_pt,ctr_pt: list, lists of coordinates for synapses.'''
    
    
    syns = nglui.parser.line_annotations(state,synapse_layer)
    if np.shape(syns)[0] != 2:
        raise Exception( print('Incorrectly formatted synapse annotation. Requires two lists: Presynapse coordinates, Postsynapse coordinates'))
           
    else:
        pre_pt = syns[0]
        post_pt = syns[1]
        ctr_pt = (np.array(pre_pt) + np.array(post_pt)) / 2
    return(pre_pt,post_pt,np.ndarray.tolist(ctr_pt))


def format_synapse(pre,post,ctr):
    ''' Format a synapse for upload to a synapse table
    Args:
           pre: list, mip0 xyz coordinates to presynapse
           post: list, mip0 xyz coordinates to postsynapse
           ctr: list, mip0 xyz coordinates to center point
    Returns:
           data: formatted dict for a synapse table annotation upload
    '''
    data = {
    "type": "synapse",
    'pre_pt': {'position': pre},
    'post_pt': {'position': post},
    'ctr_pt': {'position': ctr}

}
    return(data)

def upload_synapses(synapse_coordinates,table_name,description=None):
    ''' Add synapses to a synapse table.
    Args: 
        synapse_coordinates: list, list of shape [3,n,3]. Dim 0 is [pre,post,ctr], Dim 1 is the entry, Dim 2 is [x,y,z]. Output of get_synapses
    ##TODO: Some sort of check for redundant synapses is necessary. We could make each upload need to go to a new table, but that seems absurd. 
    '''
    try:
        client.annotation.create_table(table_name=table_name,
                                   schema_name='synapse',
                                   description=description)
    except:
        print('table exists')
        
    for i in range(np.shape(synapse_coordinates)[1]):
        try:
            entry = format_synapse(synapse_coordinates[0][i],synapse_coordinates[1][i],synapse_coordinates[2][i])
            client.annotation.post_annotation(table_name=table_name, data=[entry])
        except:
            print('Fail',entry)

#### Add a table from an annotation layer
##### There currently are not checks on uploading duplicate data, so do not go crazy with this.

In [35]:
layer_name = None
table_name = None
description = None

entries = soma_table_entries(state,layer_name=layer_name)
upload_cells(entries,table_name,description=description)

ValueError: None is not in list

#### Some methods for working around the materialization engine.
    1. Download an annotation table
    2. Look up the root_ids associated with the points
    3. Generate a dataframe that looks like the microns format (post materialization?)

In [37]:
def download_annotation_table(table_name,ids=range(1000)):
    entries = client.annotation.get_annotation(table_name,ids)
    annotation_table = pd.DataFrame(entries)
    return(annotation_table)


def seg_from_pt(pts,vol,image_res=np.array([4.3,4.3,45]),max_workers=4):
    ''' Get segment ID at a point. Default volume is the static segmentation layer for now. 
    Args:
        pts (list): list of 3-element np.arrays of MIP0 coordinates
        vol_url (str): cloud volume url
    Returns:
        list, segment_ID at specified point '''
    
    
    seg_mip = vol.scale['resolution']
    res = seg_mip / image_res

    pts_scaled = [pt // res for pt in pts]
    results = []
    with futures.ThreadPoolExecutor(max_workers=max_workers) as ex:
        point_futures = [ex.submit(lambda pt,vol: vol[list(pt)][0][0][0][0], k,vol) for k in pts_scaled]
        
        for f in futures.as_completed(point_futures):
            results=[f.result() for f in point_futures]
       
    return results



def generate_soma_table(annotation_table,
                        segmentation_version='Dynamic_V1',
                        resolution=np.array([4.3,4.3,45]),
                        token=None):
    ''' Generate a soma table used for microns analysis. This is the workaround for a materialization engine
    Args:
        annotation_table: pd.DataFrame, output from download_cell_table. Retreived from the annotation engine.
        segmentation_version: str, Currently we have 4 for FANC. Two flat segmentations ("Flat_1" and "Flat_2") and two dynamic ("Dynamic_V1/V2"). 
                              This will only work if you have a segmentations.json in your cloudvolume folder. See examples for format.
        resolution: np.array, Resolution of the mip0 coordinates of the version (not necessarily the same as the segmentation layer resolution).
                              For all but the original FANC segmentation, this will be [4.3,4.3,45]
        token: str, currently, CloudVolume requires a workaround for passing google secret tokens. This won't work unless you edit your cloudvolume 
                              file to remove the check for hexidecimal formatting of tokens. Updates should be coming to fix this. 
        '''

    soma_table = pd.DataFrame(columns=['name','cell_type',
                                       'pt_position','pt_root_id',
                                       'soma_x_nm','soma_y_nm','soma_z_nm',
                                       'found'])
    with open(Path.home() / 'cloudvolume' / 'segmentations.json') as f:
            cloud_paths = json.load(f)
    if 'Dynamic' in segmentation_version:
        cv = CloudVolume(cloud_paths[segmentation_version]['url'],agglomerate=True,use_https=True,secrets=token)
    else:
        cv = CloudVolume(cloud_paths[segmentation_version]['url'])
        
    seg_ids = seg_from_pt(annotation_table.pt_position,cv)
    
    soma_table.name = annotation_table.tag
    soma_table.pt_position = annotation_table.pt_position
    soma_table.pt_root_id = seg_ids
    soma_table.soma_x_nm = np.array([i[0] for i in annotation_table.pt_position]) * resolution[0]
    soma_table.soma_y_nm = np.array([i[1] for i in annotation_table.pt_position]) * resolution[1]
    soma_table.soma_z_nm = np.array([i[2] for i in annotation_table.pt_position]) * resolution[2]
    
    return(soma_table)



def generate_synapse_table(annotation_table,
                        segmentation_version='Dynamic_V1',
                        resolution=np.array([4.3,4.3,45]),
                        token=None):
    ''' Generate a soma table used for microns analysis. This is the workaround for a materialization engine
    Args:
        annotation_table: pd.DataFrame, output from download_cell_table. Retreived from the annotation engine.
        segmentation_version: str, Currently we have 4 for FANC. Two flat segmentations ("Flat_1" and "Flat_2") and two dynamic ("Dynamic_V1/V2"). 
                              This will only work if you have a segmentations.json in your cloudvolume folder. See examples for format.
        resolution: np.array, Resolution of the mip0 coordinates of the version (not necessarily the same as the segmentation layer resolution).
                              For all but the original FANC segmentation, this will be [4.3,4.3,45]
        token: str, currently, CloudVolume requires a workaround for passing google secret tokens. This won't work unless you edit your cloudvolume 
                              file to remove the check for hexidecimal formatting of tokens. Updates should be coming to fix this. 
        '''
     
    
    synapse_table = pd.DataFrame(columns=['id','pre_root_id','post_root_id',
                                      'cleft_vx','ctr_pt_x_nm','ctr_pt_y_nm','ctr_pt_z_nm',
                                      'pre_pos_x_vx','pre_pos_y_vx','pre_pos_z_vx',
                                      'ctr_pos_x_vx','ctr_pos_y_vx','ctr_pos_z_vx',
                                      'post_pos_x_vx','post_pos_y_vx','post_pos_z_vx'])

    with open(Path.home() / 'cloudvolume' / 'segmentations.json') as f:
            cloud_paths = json.load(f)
    if 'Dynamic' in segmentation_version:
        cv = CloudVolume(cloud_paths[segmentation_version]['url'],agglomerate=True,use_https=True,secrets=token)
    else:
        cv = CloudVolume(cloud_paths[segmentation_version]['url'])
        
    pre_ids = seg_from_pt(annotation_table.pre_pt_position,cv)
    post_ids = seg_from_pt(annotation_table.post_pt_position,cv)
    
    synapse_table.pre_root_id = pre_ids
    synapse_table.post_root_id = post_ids
    
    # TODO: This in not a stupid way. 
    synapse_table.ctr_pt_x_nm = np.array([i[0] for i in annotation_table.ctr_pt_position]) * resolution[0]
    synapse_table.ctr_pt_y_nm = np.array([i[1] for i in annotation_table.ctr_pt_position]) * resolution[1]
    synapse_table.ctr_pt_z_nm = np.array([i[2] for i in annotation_table.ctr_pt_position]) * resolution[2]
    
    synapse_table.pre_pos_x_vx = np.array([i[0] for i in annotation_table.pre_pt_position]) 
    synapse_table.pre_pos_y_vx = np.array([i[1] for i in annotation_table.pre_pt_position]) 
    synapse_table.pre_pos_z_vx = np.array([i[2] for i in annotation_table.pre_pt_position]) 
    
    synapse_table.post_pos_x_vx = np.array([i[0] for i in annotation_table.post_pt_position]) 
    synapse_table.post_pos_x_vx = np.array([i[1] for i in annotation_table.post_pt_position]) 
    synapse_table.post_pos_x_vx = np.array([i[2] for i in annotation_table.post_pt_position]) 
    
    return(synapse_table)
    
    

#### Download tables
- This will download tables from the annotation engine to then look up segment IDs and format the data

In [38]:
print(client.annotation.get_tables())
cell_table = download_annotation_table('test_T1MN_soma_table')
syn_table = download_annotation_table('test_synapse_table')

['test_table', 'test_synapse_table', 'test_synapse_table_2', 'test_T1MN_soma_table', 'T1MN_somas', '10B_somas', 'claw_neurons']


#### Convert annotation tables materialized(?) data

In [48]:
soma_table = generate_soma_table(cell_table,token=dev_token)
synapse_table = generate_synapse_table(syn_table,token=dev_token)


Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.77it/s]A

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.71it/s]

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.68it/s]



Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.59it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.92it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.54it/s]A

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.27it/s]



Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.36it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.98it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.51it/s]


Downloading: 100%|██████

In [60]:
soma_table.to_csv('/Users/brandon/Documents/Repositories/Python/Neuron_Database/')

,name,cell_type,pt_position,pt_root_id,soma_x_nm,soma_y_nm,soma_z_nm,found
0,MN_A101_T1L,NaN,"[13329, 114635, 1861]",648518346704067890,57314.7,492930.5,83745.0,NaN
1,MN_A101_T1R,NaN,"[63394, 114393, 2304]",648518346662254153,272594.2,491889.9,103680.0,NaN
2,MN_A102_T1L,NaN,"[11729, 116735, 1478]",648518346682057412,50434.7,501960.5,66510.0,NaN
3,MN_A102_T1R,NaN,"[63084, 116459, 2863]",648518346676423055,271261.2,500773.7,128835.0,NaN
4,MN_A103_T1R,NaN,"[59659, 115855, 2065]",648518346702534824,256533.7,498176.5,92925.0,NaN
...,...,...,...,...,...,...,...,...
116,MN_V402_T1L,NaN,"[23167, 87333, 2184]",648518346680460455,99618.1,375531.9,98280.0,NaN
117,MN_V501_T1L,NaN,"[27494, 86790, 2515]",648518346661954522,118224.2,373197.0,113175.0,NaN
118,MN_V501_T1R,NaN,"[54883, 84397, 2319]",648518346666755657,235996.9,362907.1,104355.0,NaN
119,MN_V601_T1R,NaN,"[49727, 85190, 2264]",648518346662417225,213826.1,366317.0,101880.0,NaN
